## Train a model

In [ ]:
from train_nep.config import get_nep_config
from train_nep.dataset import StructureDataset, collate_fn
from train_nep.model import NEP
from train_nep.alchemy import Alchemy
from torch.utils.data import DataLoader
from wizard.io import read_xyz

para = get_nep_config()
frames = read_xyz("../Repository/PbTe/train.xyz")

train_dataset = StructureDataset(frames=frames, para=para)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)

model = NEP(para)
alchemy = Alchemy(
    model=model,
    training_set=train_loader,
    save_path="../Repository/PbTe/nep_model.pt"
)
alchemy.fit(epochs=100)

## Extract Model

In [ ]:
from train_nep.model import NEP

nep = NEP.from_checkpoint("../Repository/PbTe/nep_model.pt")
nep.print_model_info()
#nep.save_to_nep_format("../Repository/PbTe/nep_torch.txt")

## Compare Model

In [ ]:
from wizard.io import read_xyz, plot_force_results
from calorine.calculators import CPUNEP
from train_nep.interface import NEPCalculator
%matplotlib inline

frames = read_xyz("../Repository/PbTe/train.xyz")
calc1 = CPUNEP("../Repository/PbTe/nep_torch.txt")
calc2 = NEPCalculator("../Repository/PbTe/nep_model.pt")
calcs = [calc1, calc2]
plot_force_results(frames, calcs, ['nep_cpu', 'torch'])